# All Library Functions
## Purpose
In this example we will demonstrate how to:
   - Build a Coreset tree on a train dataset.
   - Adding to the tree additional data through partial_build
   - Update samples' labels
   - Update samples' features
   - Remove samples
   - Compare the model trained on the Coreset with the model trained on the full dataset

In this example we'll be using the well-known Covertype Dataset (https://archive.ics.uci.edu/ml/datasets/covertype).
We will split the data to three parts:
  - train_1 - 50% of the data
  - train_2 - 20% of the data
  - test - 30% of the data


In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
import random

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from dataheroes import CoresetTreeServiceLG

## Prepare datasets

In [2]:
# Load Covertype dataset as a pandas data frame.
# In the output data frame all columns are features beside the last column.
# The last column (Cover_Type) is the target
df = fetch_covtype(as_frame=True).frame
# add random_date column
start_date = datetime.date(year=2021, month=1, day=1)
end_date = datetime.date(year=2022, month=12, day=31)
df['random_date'] = [start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
                     for _ in range(df.shape[0])]
# set random_date column as first in dataframe
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
# Split dataset: train_1 = 50%, train_2=20%, test=30%
train, test = train_test_split(df, test_size=0.3, random_state=42)
train_1, train_2 = train_test_split(train, test_size=20/70, random_state=42)

# Prepare the data directory and set the file names.
data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)
train_1_file_path = data_dir / "train_1.csv"
train_2_file_path = data_dir / "train_2.csv"
test_file_path = data_dir / "test.csv"

# Store datasets as CSV.
# We use the data frame index as the unique identifier and define a column for it (index_column)
# Defining an index column is optional. In this example we set it to have a reference to the original dataset.
train_1.to_csv(train_1_file_path, index_label="index_column")
train_2.to_csv(train_2_file_path, index_label="index_column")
test.to_csv(test_file_path, index_label="index_column")

## 1. Build the tree
Run `build_from_file` on the first train file.
It will include ~290K sample. We pass `n_classes` and `n_instances` to help the tree calculate an optimal Coreset size.

In [3]:
# Tell the tree how the data is structured.
# In this example we have an index column, a target column and a property column. 
# Property is a column you can filter on, but it isn't taken into account when building 
# the coreset or training the model. 
# All other columns are features.
data_params = {
    'properties': [
            {'name': 'random_date'},
        ],
    'target': {'name': 'Cover_Type'},
    'index': {'name': 'index_column'}
}
# Initialize the service and build the tree.
# The tree uses the local file system to store its data.
# After this step you will have a new directory .dataheroes_cache
service_obj = CoresetTreeServiceLG(data_params=data_params,
                                   optimized_for='training',
                                   n_classes=7,
                                   n_instances=406_000
                                  )
service_obj.build_from_file(train_1_file_path)

## 2. Plot the tree

In [ ]:
output_dir = Path('output')
output_dir.mkdir(exist_ok=True)
service_obj.plot(output_dir, "train_tree")

PosixPath('output/train_tree.png')

## 3. Get coreset from the tree, train a model based on it

In [5]:
# Ignore convergence warnings for logistic regression
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

# Get the top level coreset (~2K samples with weights)
coreset = service_obj.get_coreset()
indices, X, y = coreset['data']
w = coreset['w']
# Train a logistic regression model on the coreset.
coreset_model = LogisticRegression().fit(X, y, sample_weight=w)
n_samples_coreset = len(y)

## 4.1. Train a model on the full dataset for comparison
We use the same part of the dataset that was used for building the tree

In [6]:
X = train_1.iloc[:, 1:-1].to_numpy() 
y = train_1.iloc[:, -1].to_numpy()

full_dataset_model = LogisticRegression().fit(X, y)
n_samples_full = len(y)

### 4.2. Compare models quality
Test both coreset and full model on test dataset and compare their AUC scores

In [7]:
# Read test dataset from file
test_df = pd.read_csv(test_file_path)
X_test = df.iloc[:, 1:-1].to_numpy()
y_test = df.iloc[:, -1].to_numpy()

# Evaluate model
coreset_score = roc_auc_score(y_test, coreset_model.predict_proba(X_test), multi_class='ovr')
full_dataset_score = roc_auc_score(y_test, full_dataset_model.predict_proba(X_test), multi_class='ovr')

print(f"Coreset AUC score ({n_samples_coreset:,} samples): {coreset_score}")
print(f"Full dataset AUC score ({n_samples_full:,} samples): {full_dataset_score}")

Coreset AUC score (1,182 samples): 0.8326490662571097
Full dataset AUC score (290,505 samples): 0.7903846802843272


## 5.1. Save the tree for later use

In [8]:
out_dir = Path('output')
out_dir.mkdir(exist_ok=True)
save_tree_name = 'train_1_coreset_tree'
service_obj.save(out_dir, save_tree_name, override=True)

PosixPath('output/train_1_coreset_tree')

### 5.2. Load the saved tree

In [9]:
service_obj = CoresetTreeServiceLG.load(out_dir, save_tree_name)

## 6. Partial build
Add the second train dataset to the tree using `partial_build_from_file`.

In [10]:
service_obj.partial_build_from_file(train_2_file_path)

## 7. Get important samples
Call `get_important_samples` for a certain class (the biggest) for 100 samples to identify mislabeled samples.

In [11]:
# There are 7 classes, we choose the biggest one, that is 2
biggest_class = 2
number_important_samples = 100
important_samples = service_obj.get_important_samples(class_size={biggest_class: number_important_samples})

## 8. Update targets
Call `update_targets` for some returned samples above. To further compare the models' result (tree vs. full dataset) we also update the targets in the same way, for the same samples on full dataset.

In [12]:
# take first 10 samples for update (0..9)
indexes_for_update_targets = np.arange(0,10)
# updating target with different values (not equal 2)
y_new = [1, 3, 3, 3, 3, 4, 5, 6, 7, 7]
# update selected samples on the coreset tree
service_obj.update_targets(important_samples['idx'][indexes_for_update_targets], y_new)

# Same update on full dataset
train.loc[important_samples['idx'][indexes_for_update_targets], 'Cover_Type'] = y_new

## 9. Update features
Call `update_features` for some returned samples above (different ones than 8).

In [13]:
# take next 10 samples for update (10..19)
indexes_for_update_features = np.arange(10,20)
X_new = important_samples['X'][indexes_for_update_features]
# modify X, as a simple example - set all binary fields = 1
for sample in X_new:
    sample[sample == 0] = 1
# update selected samples on the coreset tree
service_obj.update_features(important_samples['idx'][indexes_for_update_features], X_new)

# same features updated on full dataset
train.loc[important_samples['idx'][indexes_for_update_features], train.columns[1:-1]] = X_new

## 10. Remove samples
Call `remove_samples` for some returned samples above (different ones than 8 and 9).

In [14]:
# take next 10 samples for remove (20..29)
indexes_for_remove = np.arange(20,30)
# remove selected samples from the coreset tree
service_obj.remove_samples(important_samples['idx'][indexes_for_remove])

# remove same samples from full dataset
train = train.drop(train.loc[important_samples['idx'][indexes_for_remove]].index)

## 11.1 Filter data from the tree
Run `filter_out_samples` to filter out all samples with elevation < 2000.

In [15]:
# index of Elevation feature
elevation_feature_index = [f.name for f in service_obj.data_manager.data_params.features].index('Elevation')
# filter function
def filter_out_elevation_less_2000(indexes, X, y, props):
    return indexes[X[:, elevation_feature_index] < 2000]
# filter samples from the tree
service_obj.filter_out_samples(filter_out_elevation_less_2000)
# Same filter applied to the full dataset
train = train[train['Elevation'] >= 2000]

## 11.2 Filter data by property
Run `filter_out_samples` to filter out all samples with random_date <= 05-01-2021 (so filtering out the first 5 days).

In [16]:
# filter function
def filter_out_dates(indexes, X, y, props):
    date_arr = np.array([datetime.datetime.strptime(d[0], '%Y-%m-%d').date() for d in props])
    return indexes[date_arr <= datetime.date(year=2021, month=1, day=5)]
# filter samples from the tree
service_obj.filter_out_samples(filter_out_dates)
# Same filter applied to the full dataset
train = train[train['random_date'] > datetime.date(year=2021, month=1, day=5)]

## 12. Fit on the tree
Run `fit` directly on the tree

In [17]:
service_obj.fit()
n_samples_coreset = len(service_obj.get_coreset()['w'])

## 13. Save coreset

In [18]:
final_coreset_path = os.path.join(output_dir, "final_coreset")
service_obj.save_coreset(final_coreset_path)

## 14.1 Train model of entire dataset
Train model of entire train dataset to compare AUC score with coreset

In [19]:
X = train.iloc[:, 1:-1].to_numpy()
y = train.iloc[:, -1].to_numpy()

full_updated_dataset_model = LogisticRegression().fit(X, y)
n_samples_full = len(y)

### 14.2 Compare models quality

In [20]:
final_tree_lg_score = roc_auc_score(y_test, service_obj.predict_proba(X_test), multi_class='ovr')
full_updated_dataset_score = roc_auc_score(y_test, full_updated_dataset_model.predict_proba(X_test), multi_class='ovr')

print(f"Updated Coreset AUC score ({n_samples_coreset:,} samples): {final_tree_lg_score}")
print(f"Full updated dataset AUC score ({n_samples_full:,} samples): {full_updated_dataset_score}")

Updated Coreset AUC score (1,108 samples): 0.806217250816925
Full updated dataset AUC score (402,999 samples): 0.7970536830030398
